In [1]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [2]:
import galois
import numpy as np
import time

In [3]:
#definisco un campo di Galois come nell'esempi
field = galois.GF(2**3)

In [4]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 3 #le righe
n_k = 4 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [25]:
#questa è con l'esempio di 4.3
A = field([[1,2,4,7],
           [0,2,3,1],
           [2,3,2,0]])

In [5]:
#ho verificato che funziona pure con i random
"""A = field(np.random.randint(0, field.order, size=(k, n_k)))
print(A)"""

[[3 3 3 1]
 [0 5 2 4]
 [7 4 7 1]]


In [6]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [7]:
A_primo = field(matrice_bin1 @ A @ matrice_bin2)
print(A_primo)

[[7 1 4 7]
 [3 1 3 3]
 [0 4 5 2]]


In [28]:
"""A_primo = field([[2,2,0,3],
           [2,0,2,3],
           [0,3,1,2]])"""

'A_primo = field([[2,2,0,3],\n           [2,0,2,3],\n           [0,3,1,2]])'

In [8]:
#definito funzione ordinamento lessicografico riga (ordino ogni riga e definisco ordine degli indici) e gestiti casi di 
#conflitto. L'argomento colonne è quello che ci indica in che ordine verificare i conflitti 
#ex: colonne=[0,1,2] --> ordino per colonna 0, se c'è conflitto tra due vedo la colonna 1 e cosi via

#faccio ordinamento crescente dei valori all'interno della riga e torno matrice con righe ordinate (ordinato).
#da questa matrice faccio l'ordinamento elemento a elemento posizionale corrispondente e definisco quindi gli indici 
#riordinati. ex: [1 0 2] --> prima la riga ORIGINALE 1 poi riga ORIGINALE 0 e così via. Per originale intendo "matrice"

#questa è la funzione che gli fornisco la matrice ordinata all'interno delle righe e lei pme la ordina per righe
def ordina_matrice(matrice):
    print(matrice)
    #righe matrice
    n = matrice.shape[0]  
    #contatore confronti
    confronti = 0  
    #array indici originali
    indici = list(np.arange(n))  
    print("Matrice ordinata:\n", matrice) #QUESTO AI FINI DI VELOCITA LO POSSO TOGLIERE

    #bubble sort per conteggio confronti
    for i in range(n):
        for j in range(i + 1, n):
            print(f"Confronto tra riga {i} e riga {j}")
            #confronto per primo el riga poi secondo e cosi via
            for k in range(matrice.shape[1]):  
                confronti += 1
                #sono ordinate tra loro, vado avanti
                if matrice[indici[i], k] < matrice[indici[j], k]: 
                    print(f"Riga {i} ({matrice[indici[i]]}) è minore di Riga {j} ({matrice[indici[j]]})")
                    break
                #devo mettere la riga sopra quella con cui è confrontata e andare avanti
                elif matrice[indici[i], k] > matrice[indici[j], k]:
                    print(f"Riga {i} ({matrice[indici[i]]}) è maggiore di Riga {j} ({matrice[indici[j]]})")
                    #sposto indice sopra
                    indici[i], indici[j] = indici[j], indici[i]
                    print("Indici dopo lo spostamento:", indici)
                    break 
                if k == (n_k - 1):
                    print("Algoritmo ha almeno due righe uguali --> fallisco")
                    return [], confronti
    print("Indici delle righe ordinate:", indici)
    print("Numero totale di confronti:", confronti)
    
    return indici, confronti



def ordina_per_riga(matrice):
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
        
    ordinato = np.array(ordinato)
    
    indici_ordinati, num_confronti = ordina_matrice(ordinato)
    if len(indici_ordinati) == 0:
        return []
    print("Matrice ordinata per righe: ",matrice[indici_ordinati])
    return matrice[indici_ordinati]

#ordino poi per il primo elemento della colonna e in caso di conflitti li ho gestiti. 
def ordina_per_colonna(matrice, righe):
    if len(matrice) == 0:
        return []
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
    return matrice[:, indici_ordinamento]

#qui faccio il test
def test_uguaglianza(matrice1, matrice2):
    righe = range(k)
    print("Matrice1: ", matrice1)

    ordinata1 = ordina_per_colonna(ordina_per_riga(matrice1), righe)
    print("\nOrdinata 1: ",ordinata1)

    print("Matrice2: ", matrice2)
    ordinata2 = ordina_per_colonna(ordina_per_riga(matrice2), righe)
    print("\nOrdinata2: ",ordinata2)     

    if np.array_equal(ordinata1, ordinata2):
        print("Le due matrici hanno la stessa CF.")
        return True
    else:
        print("Le due matrici hanno CF diversa")
        return False   



In [9]:
test_uguaglianza(A, A_primo)

Matrice1:  [[3 3 3 1]
 [0 5 2 4]
 [7 4 7 1]]
[[1 3 3 3]
 [0 2 4 5]
 [1 4 7 7]]
Matrice ordinata:
 [[1 3 3 3]
 [0 2 4 5]
 [1 4 7 7]]
Confronto tra riga 0 e riga 1
Riga 0 ([1 3 3 3]) è maggiore di Riga 1 ([0 2 4 5])
Indici dopo lo spostamento: [np.int64(1), np.int64(0), np.int64(2)]
Confronto tra riga 0 e riga 2
Riga 0 ([0 2 4 5]) è minore di Riga 2 ([1 4 7 7])
Confronto tra riga 1 e riga 2
Riga 1 ([1 3 3 3]) è minore di Riga 2 ([1 4 7 7])
Indici delle righe ordinate: [np.int64(1), np.int64(0), np.int64(2)]
Numero totale di confronti: 4
Matrice ordinata per righe:  [[0 5 2 4]
 [3 3 3 1]
 [7 4 7 1]]

Ordinata 1:  [[0 2 4 5]
 [3 3 1 3]
 [7 7 1 4]]
Matrice2:  [[7 1 4 7]
 [3 1 3 3]
 [0 4 5 2]]
[[1 4 7 7]
 [1 3 3 3]
 [0 2 4 5]]
Matrice ordinata:
 [[1 4 7 7]
 [1 3 3 3]
 [0 2 4 5]]
Confronto tra riga 0 e riga 1
Riga 0 ([1 4 7 7]) è maggiore di Riga 1 ([1 3 3 3])
Indici dopo lo spostamento: [np.int64(1), np.int64(0), np.int64(2)]
Confronto tra riga 0 e riga 2
Riga 0 ([1 3 3 3]) è maggiore di Rig

True